In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
import time
import tqdm
import sys
sys.path.append('../..')
from comparisonHC import HandlerTriplets, OracleTriplets, get_AddS_triplets, get_MulK_triplets, get_tSTE_triplets, ComparisonHC

In [2]:
#load the dataset
df = pd.read_csv("../../resources/zoo.csv")

In [3]:
df

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


In [4]:
df = df.drop('animal_name',axis=1)

In [5]:
arr = df.to_numpy()

In [6]:
#get the similarity matrix
Cos_sim = sklearn.metrics.pairwise.cosine_similarity(arr, dense_output=True)

In [7]:
#objects in the data
print(n)

101


In [8]:
adds3_rev = []
tste_rev = []
mulk_rev = []
standard_rev = []

for i in tqdm.tqdm(range(10)):
    Oracle = OracleTriplets(Cos_sim,n,n_triplets=int(n*n),proportion_noise=0.05)
    adds_similarities = get_AddS_triplets(Oracle,n)
    
    chc = ComparisonHC(adds_similarities,n)
    chc.fit([[j] for j in range(n)])
    adds3_rev.append(-chc.cost_dasgupta(adds_similarities))
    
    tste_similarities = get_tSTE_triplets(Oracle,n)
    chc_tste = ComparisonHC(tste_similarities,n)
    chc_tste.fit([[j] for j in range(n)])
    tste_rev.append(-chc_tste.cost_dasgupta(adds_similarities))
    
    mulk_similarities = get_MulK_triplets(Oracle,n)
    chc_mulk = ComparisonHC(mulk_similarities,n)
    chc_mulk.fit([[j] for j in range(n)])
    mulk_rev.append(-chc_mulk.cost_dasgupta(adds_similarities))
    
    chc_standard = ComparisonHC(Cos_sim,n)
    chc_standard.fit([[j] for j in range(n)])
    standard_rev.append(-chc_standard.cost_dasgupta(adds_similarities))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:33<00:00, 15.34s/it]


In [9]:
adds3_rev = np.array(adds3_rev)
tste_rev = np.array(tste_rev)
mulk_rev = np.array(mulk_rev)
standard_rev = np.array(standard_rev)

In [10]:
#getting mean and standard deviation of the revenue over 10 runs
adds3_mean_rev = np.mean(adds3_rev)
adds3_std_rev = np.std(adds3_rev)
tste_mean_rev = np.mean(tste_rev)
tste_std_rev = np.std(tste_rev)
mulk_mean_rev = np.mean(mulk_rev)
mulk_std_rev = np.std(mulk_rev)
cosine_mean_rev = np.mean(standard_rev)
cosine_std_rev = np.std(standard_rev)

In [11]:
print("The results are:")
print("\t Mean of Revenue using AddS-3: ",adds3_mean_rev)
print("\t Standard Deviation of Revenue using AddS-3: ",adds3_std_rev)
print("\t Mean of Revenue using t-STE: ",tste_mean_rev)
print("\t Standard Deviation of Revenue using t-STE: ",tste_std_rev)
print("\t Mean of Revenue using MulK-3: ",mulk_mean_rev)
print("\t Standard Deviation of Revenue using MulK-3: ",mulk_std_rev)
print("\t Mean of Revenue using Cosine Similarity: ",cosine_mean_rev)
print("\t Standard Deviation of Revenue using Cosine Similarity: ",cosine_std_rev)

The results are:
	 Mean of Revenue using AddS-3:  288258.5
	 Standard Deviation of Revenue using AddS-3:  5115.515325947133
	 Mean of Revenue using t-STE:  226473.2
	 Standard Deviation of Revenue using t-STE:  9632.27197290442
	 Mean of Revenue using MulK-3:  211038.7
	 Standard Deviation of Revenue using MulK-3:  16692.3088160386
	 Mean of Revenue using Cosine Similarity:  290288.1
	 Standard Deviation of Revenue using Cosine Similarity:  2173.379647001416
